# Move file together

In [2]:
import os
import shutil

In [10]:
#  20230915  20231101  20231102  20231110  20231130  20240107  20240118  20240131

FileName = "20240131"

In [14]:
# source_folder = '/home/john/network/RETFound/data/Images_Files/'
# destination_folder = '/home/john/network/RETFound/data/ImageSelectROI'

source_folder = '/home/john/network/RETFound/data/Images_Files/'
destination_folder = '/home/john/network/RETFound/data/dataset/'

In [15]:
def move_file(source,destination):
    count = 0
    for root,sub,files in os.walk(source):
        for file in files:
            source_folder_path = os.path.join(root,file)
            destination_file_path = os.path.join(destination,file)
            shutil.copy2(source_folder_path,destination_file_path)
            count += 1
    print("Image files number: ",count)

In [16]:
source = source_folder + FileName
destination = destination_folder + FileName

move_file(source,destination)

Image files number:  1090


# Format _  -> -

In [3]:
import os
import re

In [166]:
images_flle_ = '/home/john/network/RETFound/data/dataset/'

images_flle = images_flle_ + FileName

In [167]:
Images_format = re.compile(r'(\d+)-(\d+)-(L|R)-(\d+).jpg')
Images_format_ = re.compile(r'(\d+)_(\d+)_(L|R)_(\d+).jpg')

count = 0
count_ = 0
other = 0

for image in os.listdir(images_flle):
    match = Images_format.search(image)
    match_ = Images_format_.search(image)

    if match_:
        old_name = os.path.join(images_flle, image)
        new_name = os.path.join(images_flle, image.replace("_", "-"))
        os.rename(old_name, new_name)

# ROI

In [161]:
from __future__ import print_function
from multiprocessing.pool import ThreadPool

import pylab as pl
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import pandas as pd

In [175]:
# input_route = '/home/john/network/RETFound/data/ImageSelectROI/'
# output_route = '/home/john/network/RETFound/data/ImageSelectROI_output/'

input_route_ = '/home/john/network/RETFound/data/dataset/'
input_route =input_route_ + FileName + "/"

output_route = '/home/john/network/RETFound/data/A/'

In [176]:
input_route

'/home/john/network/RETFound/data/dataset/20240131/'

In [177]:
def crop_image_from_gray(img,tol=7):

    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))] 
    
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mask = gray_img>tol        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]

        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img


def circle_crop_single(img):   

    img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)

    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    
    return img 


def circle_crop(File):   
    
    InputFile = input_route + File
    img = cv2.imread(InputFile)
    img = crop_image_from_gray(img)    
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)

    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    
    return img 

In [178]:
dir_list = os.listdir(input_route)

for i in dir_list:
    ImageCut = circle_crop(i)
    output_path = os.path.join(output_route,i)
    cv2.imwrite(output_path, ImageCut)

print("Done")

Done


# outcome & Images select 找是否有檔名錯誤的檔案

In [19]:
df_csv = pd.read_csv('/home/john/network/RETFound/data/csv/202301-0616共照網個案_Wei_1130.csv')
df_csv

,病歷號,年齡,性別,fundoscope(共做過幾次),1st fundo (date),2nd fundo (date),3rd fundo (date),4rd fundo (date),5rd fundo (date),6rd fundo (date),...,運動種類,運動頻率分/次,運動頻率次/週,衛教檢驗日期,飲食,藥物認識,運動的目的及注意事項,生病時的照顧,自我監測,指導注射技巧
0,4314535,64,1,14.0,20080415,20090210.0,20100205.0,20110419.0,20120424.0,20130618.0,...,NaN,NaN,NaN,2023/03/03,NaN,NaN,NaN,NaN,NaN,NaN
1,4069957,69,1,8.0,20051215,20150506.0,20170202.0,20171225.0,20190116.0,20200422.0,...,NaN,NaN,NaN,2023/04/12,NaN,NaN,NaN,NaN,NaN,NaN
2,4386971,71,2,3.0,20200214,20210414.0,20220801.0,NaN,NaN,NaN,...,NaN,NaN,NaN,2023/01/17,NaN,NaN,NaN,NaN,NaN,NaN
3,4845785,53,1,3.0,20200811,20211202.0,20230119.0,NaN,NaN,NaN,...,NaN,NaN,NaN,2023/01/18,NaN,NaN,NaN,NaN,NaN,NaN
4,4132852,60,2,4.0,20180117,20181002.0,20200226.0,20221028.0,NaN,NaN,...,NaN,NaN,NaN,2023/05/29,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302,4241840,69,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2023/06/06,NaN,NaN,1.0,NaN,NaN,NaN
1303,1795454,62,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1304,2685764,64,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1305,3807636,56,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
columns_keep = ['病歷號','hyperlipidemia (0或1)','年齡','性別','HTN (0或1)','BMI','血壓-收縮壓 (mmHg)','血壓-輸張壓 (mmHg)','糖化血色素 (%)','發病起始年']

new_dataframe = df_csv[columns_keep]
new_dataframe = new_dataframe.rename(columns = 
                                    {'病歷號':'ID','hyperlipidemia (0或1)':'hyperlipidemia','年齡':'Age','性別':'Gender','HTN (0或1)':'HTN',
                                    '血壓-收縮壓 (mmHg)':'SP','血壓-輸張壓 (mmHg)':'DP','糖化血色素 (%)':'HbA1c','發病起始年':'Years'})

In [21]:
new_dataframe

,ID,hyperlipidemia,Age,Gender,HTN,BMI,SP,DP,HbA1c,Years
0,4314535,1,64,1,1,25.03,104.0,54.0,7.9,2001.0
1,4069957,0,69,1,0,24.96,151.0,78.0,6.7,1986.0
2,4386971,1,71,2,1,35.61,135.0,82.0,8.8,2000.0
3,4845785,1,53,1,0,27.53,123.0,66.0,9.1,2012.0
4,4132852,1,60,2,1,22.07,128.0,67.0,6.9,2017.0
...,...,...,...,...,...,...,...,...,...,...
1302,4241840,NaN,69,2,NaN,20.72,131.0,61.0,NaN,2007.0
1303,1795454,NaN,62,1,NaN,25.22,154.0,84.0,NaN,2011.0
1304,2685764,NaN,64,2,NaN,35.10,179.0,93.0,6.0,2011.0
1305,3807636,NaN,56,1,NaN,NaN,185.0,84.0,NaN,NaN


In [22]:
import warnings
warnings.filterwarnings("ignore")

In [24]:
Fudus_File = '/home/john/network/RETFound/data/ImageSelectROI_output'

# 過濾出圖片對應的 outcome 欄位

file_list = os.listdir(Fudus_File)
dataframe = pd.DataFrame(columns=df_csv.columns)

for filename in file_list:

    for i in range(0,len(filename)):
        if filename[i] == '-' :
            found = True
            break
    
    if found:
        match = False

        for j in range(0, len(df_csv['病歷號'])):
            if filename[0:i] == str(df_csv['病歷號'][j]):
                new_row = df_csv.iloc[j]
                dataframe = dataframe.append(new_row,ignore_index = True)
                match = True
                
        if not match:
                    print(f"Not found : {filename[0:i]}") 

# ID分類

In [5]:
import cv2

In [6]:
# input_directory =  '/home/john/network/RETFound/data/ImageSelectROI_output/'
# output_directory ='/home/john/network/RETFound/data/ID/'

input_directory =  '/home/john/network/RETFound/data/ROI/'
output_directory ='/home/john/network/RETFound/data/ID/'

os.makedirs(output_directory, exist_ok=True)

format = re.compile(r'(\d+)-(\d+)-(L|R)-(\d+).jpg')


for filename in os.listdir(input_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Check image is JPG or JPEG or PNG
        match = format.search(filename)

        if match:
            input_path = os.path.join(input_directory, filename)

            # Loading image
            image = cv2.imread(input_path)

            group_id = match.group(1)

        resolution_folder = os.path.join(output_directory, group_id)
        os.makedirs(resolution_folder, exist_ok=True)

        # Copy images to different file
        output_path = os.path.join(resolution_folder, filename)
        cv2.imwrite(output_path, image)

# Training validation Testing

In [7]:
import os
import random
import shutil

In [8]:
def split_folders(input_folder, Training_path,Testing_path,split_ratio=0.8):

    # 取得所有子資料夾
    subfolders = [f.path for f in os.scandir(input_folder) if f.is_dir()]

    total_folders = len(subfolders)
    split_index = int(total_folders * split_ratio)
    

    random.shuffle(subfolders)

    for i, folder_path in enumerate(subfolders):
        folder_name = os.path.basename(folder_path)
        if i < split_index:
            destination_path = os.path.join(Training_path, folder_name)
        else:
            destination_path = os.path.join(Testing_path, folder_name)
        shutil.copytree(folder_path, destination_path)

    

def second_split(input_folder,validation_path,split_ratio=0.8):

    subfolders = [f.path for f in os.scandir(input_folder) if f.is_dir()]
    total_folders = len(subfolders)
    split_index = int(total_folders * split_ratio)
    

    random.shuffle(subfolders)

    for i, folder_path in enumerate(subfolders):
        folder_name = os.path.basename(folder_path)
        if i < split_index:
            continue
        else:
            destination_path = os.path.join(validation_path, folder_name)
        shutil.move(folder_path, destination_path)


In [9]:
input_folder_path = '/home/john/network/RETFound/data/ID/'
Training_p =  '/home/john/network/RETFound/data/dataset_split/Training/'
Testing_p =  '/home/john/network/RETFound/data/dataset_split/Testing/'
validatoin_p = '/home/john/network/RETFound/data/dataset_split/validation/'

split_folders(input_folder_path, Training_p, Testing_p, split_ratio=0.8)
second_split(Training_p, validatoin_p, split_ratio=0.8)